# Ejemplo de K-Means

Implemente el algoritmo K-Means con TensorFlow, y aplíquelo para clasificar imágenes de dígitos escritos a mano. Este ejemplo utiliza la base de datos MNIST de dígitos manuscritos como ejemplos de capacitación (http://yann.lecun.com/exdb/mnist/).

Nota: Este ejemplo requiere TensorFlow v1.1.0 o superior.

In [1]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

# Ignora todas las GPUs, el random forest no se beneficia de ello.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
# Importar datos MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
full_data_x = mnist.train.images

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parametros
num_steps = 50 # Total pasos a seguir en la formación
batch_size = 1024 # El número de muestras por lote
k = 25 # El número de clusters
num_classes = 10 # Los 10 digitos
num_features = 784 # Cada imagen tiene 28x28 píxeles

# Imágenes de entrada
X = tf.placeholder(tf.float32, shape=[None, num_features])
# Etiquetas (para asignar una etiqueta a un centroide y realizar pruebas)
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

# K-Means Parametros
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine',
                use_mini_batch=True)

In [4]:
# Construir grafico KMeans
(all_scores, cluster_idx, scores, cluster_centers_initialized, init_op,
train_op) = kmeans.training_graph()
cluster_idx = cluster_idx[0] # arreglo para que cluster_idx sea una tupla
avg_distance = tf.reduce_mean(scores)

# Inicializar las variables (es decir, asignar su valor por defecto)
init_vars = tf.global_variables_initializer()

In [5]:
# Iniciar TensorFlow session
sess = tf.Session()

# Ejecutar el inicializador
sess.run(init_vars, feed_dict={X: full_data_x})
sess.run(init_op, feed_dict={X: full_data_x})

# Entrenar
for i in range(1, num_steps + 1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx],
                         feed_dict={X: full_data_x})
    if i % 10 == 0 or i == 1:
        print("Step %i, Avg Distance: %f" % (i, d))

Step 1, Avg Distance: 0.341471
Step 10, Avg Distance: 0.221609
Step 20, Avg Distance: 0.220328
Step 30, Avg Distance: 0.219776
Step 40, Avg Distance: 0.219419
Step 50, Avg Distance: 0.219154


In [7]:
# Asigna una etiqueta a cada centroide
# Contar el número total de etiquetas por centroide, usando la etiqueta de cada capacitación
# muestra al centroide más cercano (dado por'idx')
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]
# Asignar la etiqueta más frecuente al centroide
labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels_map)

# Operaciones de evaluación
# Búsqueda: centroid_id -> label
cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)
# Calcular la precisión
correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Modelo de prueba
test_x, test_y = mnist.test.images, mnist.test.labels
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Test Accuracy: 0.7127
